# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
weather_csv = "../output_data/WeatherPy"
weather_df = pd.read_csv(weather_csv)
weather_df


,Latitude,Longitude,City,Country,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Date (Unix Timestamp)
0,-55.334979,-20.318058,arraial do cabo,br,100.0,90.0,100.0,7.38,1.643704e+09
1,22.601688,-42.578374,ponta do sol,cv,96.0,81.0,96.0,9.31,1.643704e+09
2,-10.771279,120.568047,waingapu,id,90.0,63.0,90.0,18.10,1.643704e+09
3,52.149904,77.018349,aksu,kz,75.0,92.0,75.0,7.23,1.643704e+09
4,51.175621,-51.844702,bonavista,ca,4.0,58.0,4.0,19.71,1.643704e+09
...,...,...,...,...,...,...,...,...,...
560,-48.110060,-8.822993,cape town,za,0.0,46.0,0.0,9.22,1.643704e+09
561,89.362840,153.065920,chokurdakh,ru,100.0,97.0,100.0,2.28,1.643704e+09
562,-88.914381,-118.585849,punta arenas,cl,40.0,76.0,40.0,41.43,1.643704e+09
563,-29.319655,79.335822,bambous virieux,mu,75.0,78.0,75.0,18.41,1.643705e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:

gmaps.configure(api_key=g_key)

In [25]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)


(565,)

In [20]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
#Optimum vacation weather : A max temperature lower than 80 degrees but higher than 55. Wind speed less than 40 mph.
#cloudiness < 90%, humidity between 20% - 80%

vacation_df = weather_df.loc[
                            (weather_df.Humidity <= 80) &
                            (weather_df["Wind Speed (mph)"]< 40) &
                            (weather_df["Max Temp (F)"]> 55) &
                            (weather_df["Max Temp (F)"]< 80) &
                            (weather_df.Cloudiness < 90)]
vacation_df = vacation_df.dropna(how='any',axis=0) 
vacation_df


,Latitude,Longitude,City,Country,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Date (Unix Timestamp)
6,-39.959970,70.358019,mahebourg,mu,75.0,78.0,75.0,18.41,1.643704e+09
7,55.138864,-27.942489,grindavik,is,75.0,74.0,75.0,14.97,1.643704e+09
17,69.758056,-127.742919,norman wells,ca,75.0,71.0,75.0,11.50,1.643704e+09
18,-23.955637,136.527938,alice springs,au,75.0,74.0,75.0,8.05,1.643704e+09
20,-23.242855,119.155784,port hedland,au,75.0,70.0,75.0,23.02,1.643704e+09
24,-32.652662,-126.224748,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09
27,-11.052610,-118.890668,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09
42,-30.330011,-108.923696,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09
65,-14.567610,-128.448198,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09
80,-3.054989,15.554758,djambala,cg,67.0,57.0,67.0,5.66,1.643704e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = vacation_df
hotel_df["Hotel Name"]=""
hotel_df

,Latitude,Longitude,City,Country,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Date (Unix Timestamp),Hotel Name
6,-39.959970,70.358019,mahebourg,mu,75.0,78.0,75.0,18.41,1.643704e+09,
7,55.138864,-27.942489,grindavik,is,75.0,74.0,75.0,14.97,1.643704e+09,
17,69.758056,-127.742919,norman wells,ca,75.0,71.0,75.0,11.50,1.643704e+09,
18,-23.955637,136.527938,alice springs,au,75.0,74.0,75.0,8.05,1.643704e+09,
20,-23.242855,119.155784,port hedland,au,75.0,70.0,75.0,23.02,1.643704e+09,
24,-32.652662,-126.224748,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09,
27,-11.052610,-118.890668,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09,
42,-30.330011,-108.923696,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09,
65,-14.567610,-128.448198,rikitea,pf,59.0,68.0,59.0,8.37,1.643704e+09,
80,-3.054989,15.554758,djambala,cg,67.0,57.0,67.0,5.66,1.643704e+09,


In [55]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # make request and print url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params = params)
    
   # convert to json
    name_address = name_address.json()
    
    
    
    #incorporate a try-except to skip any that are missing a data point.
    try:
        print(name_address["results"])
        print("----------------------------------- NEXT--------------------")
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

        


[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
-----------------

[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
----------------------------------- NEXT--------------------
Missing field/result... skipping.
[]
-----------------

In [58]:
hotel_df
hotelnona_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=["Hotel Name"], inplace=False)

hotelnona_df.head()

,Latitude,Longitude,City,Country,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Date (Unix Timestamp),Hotel Name
105,-37.306331,142.048511,hamilton,au,61.0,64.0,61.0,17.16,1.643704e+09,Ferguson's Camp
181,29.649583,80.654584,dharchula,in,63.0,43.0,63.0,6.85,1.643704e+09,Amrit Hotel and Lodge


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_points = [info_box_template.format(**row) for index, row in hotelnona_df.iterrows()]
locations2 = hotelnona_df[["Latitude", "Longitude"]]
humidity2=hotelnona_df["Humidity"]

In [52]:
# Add marker layer ontop of heat map


fig = gmaps.figure()
markers = gmaps.marker_layer(locations2, info_box_content=hotel_points)
fig.add_layer(markers)

heat_layer = gmaps.heatmap_layer(locations2, weights=humidity2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))